# Exercise 1.4.2 - Building Custom CNNs
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives

* Use the TensorFlow Keras [Sequential API](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) to create a convolutional neural network ([CNN](https://en.wikipedia.org/wiki/Convolutional_neural_network));
* Train the custom CNN model on the German Traffic Sign Recognition Benchmark [GTSRB](https://benchmark.ini.rub.de/gtsrb_dataset.html) dataset.

## 1. Introduction

In [ ]:
### Importing the required modules

In [ ]:
import logging
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image_dataset_from_directory

### 1.1. Convolutional Neural Networks (CNNs)

#### Background

A [convolutional neural network](https://en.wikipedia.org/wiki/Convolutional_neural_network) ("ConvNet") is a special type of neural network particularly well-suited to handle image data. Like the feedforward neural networks we saw in [Exercise 1.3.3](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/1-Object-Detection-in-Urban-Environments/Exercises/1-3-3-Image-Classification-FNNs/2022-09-05-Image-Classification-Feed-Forward-Neural-Networks.ipynb), CNNs are composed of layers of neurons that have trainable weights and biases. In addition, CNNs often use activation functions and fully-connected layers similar to FNNs (e.g., softmax).

With CNNs, we make the assumption that we will always be using image data as our inputs. Since image data is multi-dimensional, resizing these $\left[H, W, D\right]$ arrays to flattened $\left[1x\left(H*W*D\right)\right]$ vectors is necessary when using fully-connected layers, such as in feedforward neural networks. With ConvNets, however, we make use of locally-connected layers that operate over small portions of _volumetric data_. That is, ConvNets allow us to preserve the existing structure of image data and drastically reduce the amount of trainable parameters needed in each layer of our network. For example, a single fully-connected neuron will require a number of weight values equal to the number of attributes in our image input. For an image of size $\left[64, 64, 3\right]$, we obtain $(64*64*3) = 12288$ weight values _per neuron_. For a layer with only $n = 10$ fully-connected neurons, the number of trainable weights we have skyrockets to an astonishing $n * 12288 = 122880$ parameters per layer. We will see in this exercise that by limiting the connectivity of neurons between layers, we can design a network that reduces the number of parameters required while simultaneously _increasing_ the number of layers in the architecture.

#### Layers

Most, if not all, convolutional neural network architectures use some combination of the following three distinct layer types: _convolutional layer_, _pooling layer_, and _fully-connected layer_. We have implemented fully-connected layers in [Exercise 1.3.3](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/1-Object-Detection-in-Urban-Environments/Exercises/1-3-3-Image-Classification-FNNs/2022-09-05-Image-Classification-Feed-Forward-Neural-Networks.ipynb) for feedforward neural networks, and their application in ConvNets is no different. However, we introduce the _convolutional layer_ in this exercise. We will touch on these in more detail in just a minute.

##### Input layer

It is important to first note that our _input layer_ corresponds directly to the dimensionality of our image data. One major assumption we make with convolutional neural networks is that our input image dimensions are constant. That is, that every image passing through our network shares the same _height_, _width_, and _channel_ dimensions. In ConvNets, the _depth_ of an image (its colour channels, e.g., R, G, B) is maintained throughout the network while the height and width might change. For convolutional and pooling layers, this matters because each kernel and filter will require an extra dimension $D$ equal to the number of colour channels in an image in addition to their width and height attributes. For an RGB image, a kernel/filter of size $\left(h \ x \ w\right)$ will therefore have dimensions $\left(h \ x \ w \ x \ 3\right)$.

##### Convolutional layer

Very similar to the pooling layers we studied in [Exercise 1.4.1](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/1-Object-Detection-in-Urban-Environments/Exercises/1-4-1-Pooling-Layers-CNNs/2022-09-07-Pooling-Layers-Convolutional-Neural-Networks.ipynb), these layers take an input, the volumetric image data, and perform sliding window operations ([convolutions](https://en.wikipedia.org/wiki/Convolution)) over the image. The convolution layer, like pooling layers, have _stride_, _kernel size_ and _padding_ attributes. We also introduce a _filter_ hyperparameter which allows us to specify the desired dimensionality of our convolution layer output. Convolutional layers rarely ever have just _one filter_. In fact, it is very common to see a single convolutional layer have anywhere from 32 filters up to 512. Each _filter_ represents a _feature_ in the input image (hence the output name "feature map"). These filters are templates which, when convolved with an input image, help locate and identify vertical, horizontal edges, arches, diagonal lines, etc.

The convolutional layer in a network is often directly followed by an element-wise activation layer such as the Rectified Linear Unit ([ReLU](https://en.wikipedia.org/wiki/Rectifier_\(neural_networks\))) to introduce further non-linearities into the network. Note that the activation layer's output dimensions remain constant.

##### Pooling layer

Pooling layers, as mentioned in the previous Exercise 1.4.1, reduce the size (spatial dimensions) of the images through a downsampling operation. By "summarising" neighbouring pixel values with e.g., _max_, _min_, or _average_ calculations we are able to transform the output and effectively manipulate the [receptive field](https://en.wikipedia.org/wiki/Receptive_field#In_the_context_of_neural_networks) of the image (the region within an image of which a particular neuron is sensitive to). By shrinking the image, we are essentially _increasing_ the amount of its pixels (information) we can fit into a single "patch" for a neuron to consider.

As the image becomes reduced in size as it passes through the network, neurons in deeper layers get to "see" larger and larger parts of the whole image. In turn, we can increase the receptive field of a network exponentially while increasing the number of parameters linearly. This trade-off is what affords us the ability to swap only a few fully-connected layers for _handfuls_ of convolutional stacks without increasing complexity or decreasing the receptive field.

##### Fully-connected layer

While we aren't able to eliminate fully-connected layers altogether, their use in convolutional neural networks is limited. Fully-connected layers appear at the end of a network for classification; the number of neurons of the last fully-connected layer in a CNN equals the number of distinct classes predicted. Practitioners may also choose to implement several preceding fully-connected layers with an arbitrarily large number of neurons, e.g., 512, 256, 64.

#### In summary
   * Inputs to a ConvNet are volumetric image data with fixed dimensions;
   * We use pooling layers to _downsample_ images (reduce their spatial dimensions);
   * By reducing the spatial dimensions, we _increase_ the neurons' receptive fields (how much of the image they can "see");
   * With convolutional networks we are able to eventually cover the entire region of an input image at a cost significantly reduced to fully-connected layers;

We have now learned the benefits of convolutional architectures to more-traditional neural networks (specifically, feedforward neural networks); by adding "stacks" of convolutional/pooling layers, we can exploit the complexity-efficiency trade-off. Convolutional stacks help us set reasonable limits on neural network complexity by decreasing the number of overall trainable parameters we have to work with layer-for-layer. This not only improves training efficiency but also helps boost performance on image classification tasks.

While several other CNN architectures make use of even greater improvements, such as _dropout_/_inverted dropout_ and _batch normalisation_, we will limit our discussion on those topics for now.

### 1.2. LeNet

## Details

Using the Keras API, you have to create a small convolutional neural networks using less than 15 layers, containing at least one convolutional layer, one pooling layer and one dense (fully connected layer). You can find a list of the different layers available [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers).

In [ ]:
### From Udacity's `training.py`

In [ ]:
def create_network():
    net = tf.keras.models.Sequential()
    # IMPLEMENT THIS FUNCTION
    return net

You should experiment with different designs (number of layers, types of pooling, filter sizes, number of fully connected layers, number of neurons).

You will need to feed the image directory to `training.py` (`GTSRB/Final_Training/Images/`) with `-d`, and can view the final metrics visualization in the Desktop.

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def get_module_logger(mod_name):
    logger = logging.getLogger(mod_name)
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)
    return logger

In [ ]:
### From Udacity's `training.py`

In [ ]:
logger = get_module_logger(__name__)
parser = argparse.ArgumentParser(description='Download and process tf files')
parser.add_argument('-d', '--imdir', required=True, type=str,
                    help='data directory')
parser.add_argument('-e', '--epochs', default=10, type=int,
                    help='Number of epochs')
args = parser.parse_args()    

logger.info(f'Training for {args.epochs} epochs using {args.imdir} data')

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def process(image,label):
    """ small function to normalize input images """
    image = tf.cast(image/255. ,tf.float32)
    return image,label

In [ ]:
def get_datasets(imdir):
    """ extract GTSRB dataset from directory """
    train_dataset = image_dataset_from_directory(imdir, 
                                       image_size=(32, 32),
                                       batch_size=32,
                                       validation_split=0.2,
                                       subset='training',
                                       seed=123,
                                       label_mode='int')

    val_dataset = image_dataset_from_directory(imdir, 
                                        image_size=(32, 32),
                                        batch_size=32,
                                        validation_split=0.2,
                                        subset='validation',
                                        seed=123,
                                        label_mode='int')
    train_dataset = train_dataset.map(process)
    val_dataset = val_dataset.map(process)
    return train_dataset, val_dataset

In [ ]:
### From Udacity's `training.py`

In [ ]:
# get the datasets
train_dataset, val_dataset = get_datasets(args.imdir)

In [ ]:
model = create_network()

model.compile(optimizer='adam',
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])
history = model.fit(x=train_dataset, 
                    epochs=args.epochs, 
                    validation_data=val_dataset)

In [ ]:
### From Udacity's `utils.py`

In [ ]:
def display_metrics(history):
    """ plot loss and accuracy from keras history object """
    f, ax = plt.subplots(1, 2, figsize=(15, 5))
    ax[0].plot(history.history['loss'], linewidth=3)
    ax[0].plot(history.history['val_loss'], linewidth=3)
    ax[0].set_title('Loss', fontsize=16)
    ax[0].set_ylabel('Loss', fontsize=16)
    ax[0].set_xlabel('Epoch', fontsize=16)
    ax[0].legend(['train loss', 'val loss'], loc='upper right')
    ax[1].plot(history.history['accuracy'], linewidth=3)
    ax[1].plot(history.history['val_accuracy'], linewidth=3)
    ax[1].set_title('Accuracy', fontsize=16)
    ax[1].set_ylabel('Accuracy', fontsize=16)
    ax[1].set_xlabel('Epoch', fontsize=16)
    ax[1].legend(['train acc', 'val acc'], loc='upper left')
    plt.show()

In [ ]:
display_metrics(history)

## Tips

A good starting point for small networks is LeNet5. You will find many existing implementations online.

Don't forget the basic structure of a convnet: convolutional layer, activation and pooling.

You can use the [`summary`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#summary) method of the Keras model API to print the description of your model.